In [9]:
import pandas as pd

In [10]:
# 1. Load the data
df = pd.read_csv('../data/mindx test dataset.csv')

In [11]:
# 2. Define "Efficiency" (Fuel burned per Nautical Mile)
# Formula: Efficiency = Fuel / Distance
df['fuel_per_mile'] = df['fuel_consumption'] / df['distance']

In [12]:
# Analysis 1: The "Acute" Anomaly (Bad Voyage)
# Filter for Calm weather to isolate mechanical/physical issues
calm_df = df[df['weather_conditions'] == 'Calm']
worst_voyage = calm_df.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [13]:
# ANALYSIS 2: The "Aggregate" Anomaly (Bad Ship)
# Group by Ship ID and calculate average efficiency
ship_performance = df.groupby('ship_id')['fuel_per_mile'].mean().reset_index()
worst_ship_overall = ship_performance.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [14]:
# Baselines
fleet_avg = df['fuel_per_mile'].mean()
chronic_dev = (worst_ship_overall['fuel_per_mile'] - fleet_avg) / fleet_avg * 100

In [15]:
print(f"ACUTE ANOMALY REPORT (Single Voyage)")
print(f"Vessel:      {worst_voyage['ship_id']}")
print(f"Route:       {worst_voyage['route_id']}")
print(f"Efficiency:  {worst_voyage['fuel_per_mile']:.2f} kg/nm")
print(f"Weather:     {worst_voyage['weather_conditions']}")
print("-" * 30)
print(f"CHRONIC ANOMALY REPORT (Overall Ship)")
print(f"Vessel:      {worst_ship_overall['ship_id']}")
print(f"Avg Eff:     {worst_ship_overall['fuel_per_mile']:.2f} kg/nm")
print(f"Deviation:   +{chronic_dev:.1f}% worse than fleet avg")

ACUTE ANOMALY REPORT (Single Voyage)
Vessel:      NG117
Route:       Lagos-Apapa
Efficiency:  49.91 kg/nm
Weather:     Calm
------------------------------
CHRONIC ANOMALY REPORT (Overall Ship)
Vessel:      NG108
Avg Eff:     43.53 kg/nm
Deviation:   +52.0% worse than fleet avg
